## **Implementation of multi-layer neural network using NumPy**

**Import libraries**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time

**Network parameters**

In [32]:
no_of_samples = 10000
no_of_columns = 160
no_of_classes = 2
initial_weights_range = 0.075
learning_rate = 0.05
batch_size = 20
train_index = int(0.9 * no_of_samples / batch_size) * batch_size
test_end_index = int(no_of_samples / batch_size) * batch_size
neurons_cnt_list =   list((128, 64, no_of_classes)) # format of the list: layer_1, layer_2,.....,layer_n, no_of_classes. For example (512, 128, 128, no_of_classes)

**Crate dataset**

In [24]:
np.random.seed(1)
normal_sample = np.random.normal(loc=0, scale=1,size= (no_of_samples,no_of_columns))

normal_sample_class = np.where( np.logical_or( (0.6 * normal_sample[:,4].reshape(no_of_samples,1) + 1.2 * normal_sample[:,2].reshape(no_of_samples,1)) > 1.5,
                                             (0.5 * normal_sample[:,0].reshape(no_of_samples,1) + normal_sample[:,1].reshape(no_of_samples,1)) < -0.33), np.ones((no_of_samples,1 ), dtype = 'int64'), np.zeros((no_of_samples,1 ), dtype = 'int64')  )

normal_sample_class_dumm = pd.get_dummies(normal_sample_class[:,0]).values

**Model definition**

In [25]:
def create_layers_dim_list(tmp_neuron_list, elem_to_add, acc, tmp_no_of_classes):
        if len(tmp_neuron_list) > 0:
                acc.append((elem_to_add, tmp_neuron_list[0]))
                return create_layers_dim_list(tmp_neuron_list[1:], tmp_neuron_list[0], acc, tmp_no_of_classes)
        else:
                return acc
 
    
def activation_function(x):
        return 1/(1+np.exp(-x))

def activation_derivative(x):
           return x*(1-x)


class NeuralNetworkModel:
    


    def __init__(self,  tmp_neruons_list, tmp_no_of_cols, tmp_no_of_classes, tmp_initial_weights_range, seed):
        np.random.seed(seed)
        self.biases_list = [ (2 * tmp_initial_weights_range * np.random.rand(  1 ,  curr_elem )) - tmp_initial_weights_range for curr_elem in  tmp_neruons_list]
        self.dim_list = create_layers_dim_list( tmp_neruons_list, tmp_no_of_cols, [], tmp_no_of_classes)
        self.weights_list = [ (2 * tmp_initial_weights_range * np.random.rand(curr_elem[0], curr_elem[1])) - tmp_initial_weights_range for curr_elem in self.dim_list]
        self.no_of_layers = len(tmp_neruons_list)

    def propagate_activations(self, tmp_input_data, tmp_layer_no, tmp_total_layer_cnt, tmp_act_input_list, tmp_act_list):
        if tmp_layer_no < tmp_total_layer_cnt:
            tmp_act_input_list.append(np.dot(tmp_input_data, self.weights_list[tmp_layer_no]) + self.biases_list[tmp_layer_no])
            tmp_act_list.append(activation_function (tmp_act_input_list[tmp_layer_no])) 
            return self.propagate_activations(tmp_act_list[tmp_layer_no], tmp_layer_no + 1, tmp_total_layer_cnt, tmp_act_input_list, tmp_act_list )
        else: 
            return (tmp_act_input_list, tmp_act_list)

    def error_backpropagation(self, tmp_delta_l_up, tmp_current_layer, tmp_learning_rate, tmp_input_data, tmp_act_list):
        if tmp_current_layer > 0:
            tmp_delta = np.dot(  tmp_delta_l_up,  self.weights_list[tmp_current_layer].T ) * activation_derivative(tmp_act_list[tmp_current_layer-1] )
            self.weights_list[tmp_current_layer] -= ((tmp_learning_rate) * np.dot(tmp_act_list[tmp_current_layer-1].T,  tmp_delta_l_up) )
            self.biases_list[tmp_current_layer] -= ((tmp_learning_rate) * np.sum(tmp_delta_l_up, axis = 0))
            self.error_backpropagation(tmp_delta, tmp_current_layer - 1, tmp_learning_rate, tmp_input_data, tmp_act_list)
        else:
            self.weights_list[tmp_current_layer] -= ((tmp_learning_rate) * np.dot(tmp_input_data.T,  tmp_delta_l_up) )
            self.biases_list[tmp_current_layer] -= ((tmp_learning_rate) * np.sum(tmp_delta_l_up, axis = 0))


    def process_batch(self, tmp_batch_indicies, tmp_train2, tmp_class2):
        tmp_train = tmp_train2[tmp_batch_indicies]
        tmp_class = tmp_class2[tmp_batch_indicies]
        activations_list = self.propagate_activations( tmp_train, 0, self.no_of_layers, [], [])[1]
        error_last_layer =   activations_list[self.no_of_layers -1] - tmp_class
        delta_last_layer = error_last_layer * activation_derivative(activations_list[self.no_of_layers -1] )
        self.error_backpropagation (delta_last_layer, self.no_of_layers -1, (learning_rate / tmp_train.shape[0]), tmp_train, activations_list)


    def process_one_epoch(self,  tmp_train_index, tmp_tr, tmp_cls ):
        batch_cntrl_matrix = np.arange(tmp_train_index).reshape(-1,batch_size)
        np.apply_along_axis( self.process_batch, 1 , batch_cntrl_matrix, tmp_tr, tmp_cls )
    
    
    def fit(self, tmp_train_data, tmp_class_data, tmp_train_index1, no_of_epochs):
        for _ in range(no_of_epochs):
            self.process_one_epoch( tmp_train_index1, tmp_train_data, tmp_class_data)
        
    
    def predict(self, tmp_test_data):
        return np.argmax((self.propagate_activations( tmp_test_data, 0, self.no_of_layers, [], [])[1])[self.no_of_layers -1] ,axis=1)


**Create an instance of the model**

In [33]:
tmpModel = NeuralNetworkModel(neurons_cnt_list, no_of_columns, no_of_classes, initial_weights_range, seed = 1)

**Train**

In [34]:
tmpModel.fit(normal_sample, normal_sample_class_dumm , train_index, no_of_epochs = 150)

**Predict**

In [35]:
pd.crosstab(normal_sample_class[train_index:test_end_index,0], tmpModel.predict( normal_sample[train_index:test_end_index ,:]))

col_0,0,1
row_0,,
0,473,30
1,84,413
